# Agent Evaluation
In this notebook we build an evaluation pipeline to test our model in different scenarios:
- Retrieving data from the database
- Searching Wikipedia
- Searching Youtube

## Installs and imports

In [10]:
# load env vars
import os
from dotenv import load_dotenv
load_dotenv('chromadb_evaluation.env', override=True)  # change project evaluation name in environment

if os.environ.get('OPENAI_API_KEY', ''):
    print('OPENAI_API_KEY is set')
print('Langchain project:', os.environ.get('LANGCHAIN_PROJECT', ''))

OPENAI_API_KEY is set
Langchain project: ai-librarian-database-calls-evaluation


## Get our langchain model
Let's create an instance of our langchain model that will be used in the project

In [ ]:
# create an agent
import tools
from ai_librarian import AiLibrarian

avatar = AiLibrarian()

# restrict the agent to only use these tools
agent_tools = [
    tools.search_database_for_book_information,
    # tools.search_database_for_book_reviews,
]

avatar.create_worker_agent(agent_tools)

# test the agent
# avatar.generate_model_answer('Tell me about The Three Musketeers')


## Create Evaluator

In [12]:
from langchain_openai import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

### Answer Score
Let's rate the answer of our model compared to the user query


In [13]:
def answer_score(query: str, context: str) -> bool:
    
    prompt_text = """
    You are an evaluator for a conversational AI system. The user asked:
    "{query}"
    
    The AI responded with:
    "{context}"
    
    Please grade on a scale from 0 to 5 if the response is 
    - Relevant to the question and answers the question
    - A sensible, non-recursive, and complete answer
    - Grammatically correct
    - Engaging and informative
    - Free of factual errors
    - Free of offensive content

    Answer with a single integer from 0 to 5, 0 being the worst and 5 being the best score.
    """

    prompt = ChatPromptTemplate.from_messages(
        [("system", prompt_text),]
    )

    llm = ChatOpenAI(
        temperature=0,
        model_name='gpt-4o-mini',
        api_key=os.getenv("OPENAI_API_KEY")
    )

    chain = create_stuff_documents_chain(llm, prompt)
    document = Document(page_content=context)
    return chain.invoke({"context": [document], "query": query})

# query = 'Tell me about The Three Musketeers'
# context = avatar.generate_model_answer(query)
# print('ANSWER SCORE:', answer_score(query, context))

# context = 'The three musketeers is a cartoon with dogs for children'
# print('ANSWER SCORE:', answer_score(query, context))

### Sanity Check
A sanity check is a simple check to ensure that the agent’s response is not nonsensical, empty, or wildly irrelevant. For instance, a response should not be blank, too short, or completely unrelated.

In [14]:
def sanity_check(query: str, context: str) -> bool:
    
    prompt_text = """
    You are an evaluator for a conversational AI system. The user asked:
    "{query}"
    
    The AI responded with:
    "{context}"
    
    Please determine if the response is:
    - Relevant to the question
    - A sensible, non-recursive, and complete answer

    Answer with only "True" if it passes the sanity check and "False" if it does not.
    """

    prompt = ChatPromptTemplate.from_messages(
        [("system", prompt_text),]
    )

    llm = ChatOpenAI(
        temperature=0,
        model_name='gpt-4o-mini',
        api_key=os.getenv("OPENAI_API_KEY")
    )

    chain = create_stuff_documents_chain(llm, prompt)
    document = Document(page_content=context)
    return chain.invoke({"context": [document], "query": query})

# query = 'Tell me about The Three Musketeers'
# context = avatar.generate_model_answer(query)
# print('SANITY CHECK:', sanity_check(query, context))

# context = 'The three musketeers is a cartoon with dogs for children'
# print('SANITY CHECK:', sanity_check(query, context))

### Response Time
Let's create a method to calculate the response time

In [15]:
import time

def generate_model_answer_timed(query: str) -> tuple[str, float]:
    start = time.time()
    answer = avatar.generate_model_answer(query)
    end = time.time()
    response_time = end - start
    return answer, response_time

# query = 'Tell me about the Three Musketeers'
# answer, gen_time = generate_model_answer_timed(query)
# print(answer)
# print(f"Response time: {gen_time} seconds")

### Evaluate Readability
The Flesch Reading Ease score is a popular readability test used to evaluate how easy a text is to read. It’s particularly useful in ensuring your chatbot's responses are clear and accessible to your target audience.

Score Interpretation
The Flesch Reading Ease Score ranges from 0 to 100, with higher scores indicating easier readability. Here’s a breakdown:

Score Range	Description	Readability Level  
90 - 100	Very Easy	5th grade  
80 - 89	Easy	6th grade  
70 - 79	Fairly Easy	7th grade  
60 - 69	Standard	8th-9th grade  
50 - 59	Fairly Difficult	High school  
30 - 49	Difficult	College  
0 - 29	Very Confusing	College graduate and above  


In [16]:
from textstat import flesch_reading_ease

def readability_score(text: str) -> float:
    # Higher scores indicate easier readability
    return flesch_reading_ease(text)

# query = 'Tell me about the Three Musketeers'
# answer = avatar.generate_model_answer(query)
# score = readability_score(answer)
# print(f"Readability score: {score}")

### Response length
Let's see how long an answer is in terms of characters, words and tokens

In [17]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")  # for gpt4o models

def answer_length(text: str) -> int:
    # initialise the tokenizer

    characters = len(text)
    words = len(text.split())
    tokens = len(encoding.encode(text))

    return characters, words, tokens

# query = 'Tell me about the Three Musketeers'
# answer = avatar.generate_model_answer(query)
# length = answer_length(answer)
# print(f"Character count: {length[0]}")
# print(f"Word count: {length[1]}")
# print(f"Token count: {length[2]}")

### Lexical Overlap

In [18]:
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import word_tokenize
import numpy as np
from nltk.corpus import stopwords

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_overlap(query: str, answer: str) -> float:
    # Tokenize and filter stop words
    stopwords_en = stopwords.words('english')
    query_words = set([word for word in word_tokenize(query.lower()) if word not in stopwords_en])
    answer_words = set([word for word in word_tokenize(answer.lower()) if word not in stopwords_en])
    
    # Compute embeddings
    query_embedding = model.encode(' '.join(query_words))
    answer_embedding = model.encode(' '.join(answer_words))
    
    # Calculate cosine similarity
    similarity_score = util.cos_sim(query_embedding, answer_embedding).item()
    
    return similarity_score

# Example usage
# query = 'Tell me about the Three Musketeers'
# answer = avatar.generate_model_answer(query)
# semantic_score = semantic_overlap(query, answer)
# print(f"Semantic overlap score: {semantic_score}")


### Entity Agreement
NOT WORKING

In [19]:
# import nltk
# nltk.download('maxent_ne_chunker_tab')
# nltk.download('words')
# from nltk import ne_chunk, pos_tag, word_tokenize

# def extract_entities(text):
#     # Extract named entities from text
#     chunks = ne_chunk(pos_tag(word_tokenize(text)))
#     print(chunks.leaves())
#     entities = [" ".join([token for token, _ in chunk.leaves()])
#             for chunk in chunks if hasattr(chunk, 'label')]
#     print(f"Extracted entities from '{text}': {entities}")  # Debug output
#     return entities

# def entity_agreement(query, answer):
#     # Extract entities in answer and query
#     query_entities = set(extract_entities(query.lower()))
#     answer_entities = set(extract_entities(answer.lower()))

#     print(f"Entities in query: {query_entities}")  # Debug output
#     print(f"Entities in answer: {answer_entities}")  # Debug output

#     # Calculate proportion of entities in answer that are also in query
#     if not answer_entities:
#         return 0
#     return len(answer_entities.intersection(query_entities)) / len(answer_entities)

# Example Usage
# query = 'Tell me about the Three Musketeers'
# answer = 'The Three Musketeers is a novel by Alexandre Dumas.'
# # answer = avatar.generate_model_answer(query)
# agreement_score = entity_agreement(query, answer)
# print(f"Entity Agreement Score: {agreement_score}")


## Evaluation pipeline
Let's group all the tests into one evaluation pipeline and run it on samples

In [20]:
# create an empty dataframe
import os
import pandas as pd
from datetime import datetime

# create evaluatoin folder if it does not exist
os.makedirs('evaluation', exist_ok=True)
separator = 100 * '-'

def save_row_to_csv(file: str, row: dict) -> None:

    # check if the file exists
    if os.path.exists(file):
        # load the existing dataframe
        df = pd.read_csv(file)
    else:
        # create a new dataframe
        df = pd.DataFrame(columns=[
            'query',
            'answer',
            'response_time',
            'answer_score',
            'sanity_check',
            'readability_score',
            'character_count',
            'word_count',
            'token_count',
            'semantic_overlap',
        ])

    # add the new row to the dataframe
    df.loc[len(df)] = row

    # save the dataframe to a csv file
    df.to_csv(file, index=False)


def run_evaluation(queries: list[str], title: str = 'evaluation_results') -> None:
    """Run the evaluation for a list of queries

    Args:
        queries (list[str]): List of queries to evaluate
    """

    csv_file = f'evaluation/{title}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'

    counter = 0
    n_queries = len(queries)

    for query in queries:
        
        # print progress
        counter += 1
        print(f"{separator}\nProgress: {counter}/{n_queries} | Query: {query}\n{separator}")

        # generate answer and run all the tests
        try:
            answer, response_time = generate_model_answer_timed(query)
            score = answer_score(query, answer)
            sanity = sanity_check(query, answer)
            readability = readability_score(answer)
            characters, words, tokens = answer_length(answer)
            semantic = semantic_overlap(query, answer)
        except Exception as e:
            print(f"Error with query {query}: {e}")
            continue

        # store results in a dictionary
        row = {
            'query': query,
            'answer': answer,
            'response_time': response_time,
            'answer_score': score,
            'sanity_check': sanity,
            'readability_score': readability,
            'character_count': characters,
            'word_count': words,
            'token_count': tokens,
            'semantic_overlap': semantic,
        }

        # add row to csv file
        save_row_to_csv(csv_file, row)



## Evaluation 1: Agent calls to the database
In this first evaluation, we will ask the agent for information about books that are already in the database. We will therefore evaluate how it retrieves information from the database only.

Note: The agent is fetching information from the book_info collection using `vector_db.search_collection`. It filters the result based on distance values and the agent summarizes the results in its answer.

In [21]:
# create a new langchain project to track it
title = 'database_calls_evaluation'


In [22]:
# load 100 books from the dataset we loaded into the database

# load the clean dataset into a dataframe
df = pd.read_csv('datasets/BooksDatasetClean.csv')

# get random titles and store them in a list
n_titles = 10
queries = df.sample(n=n_titles)['Title'].tolist()
print(queries)


["Make 'Em Laugh: The Funny Business of America", 'Rules of Engagement', "Parade of Popular Hits (Reader's Digest Songbook)", 'The Art of the Infinite: The Pleasures of Mathematics', 'The Devil Wears Prada a Novel', 'Fiasco: The American Military Adventure in Iraq', 'Organized Living', "The Spirit to Serve: Marriott's Way", 'The White Bull', 'The Wheel of Darkness (Pendergast)']


In [23]:
# run the evaluation
run_evaluation(queries, title)

----------------------------------------------------------------------------------------------------
Progress: 1/10 | Query: Make 'Em Laugh: The Funny Business of America
----------------------------------------------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `search_database_for_book_information` with `{'query_text': "Make 'Em Laugh: The Funny Business of America"}`




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [["Make 'Em Laugh: The Funny Business of America - From the most popular routines and the most ingenious physical shtick to the snappiest wisecracks and the most biting satire of the last century, MAKE 'EM LAUGH illuminates who we are as a nation by exploring what makes us laugh, and why. Authors Laurence Maslon and Michael Kantor draw on countless sources to chronicle the past century of American comedy and the geniuses who created and performed it-melding biography, American history, and a lotta laughs into an exuberant, important book.  Each of the six chapters focuses a different style or archetype of comedy, from the slapstick pratfalls of Buster Keaton and Lucille Ball through the wiseguy put-downs of Groucho Marx and Larry David, to the incendiary bombshells of Mae West and Richard Pryor . And at every turn the significance of these comedians-smashing social boundaries, challenging the definition of good taste, speaking the truth to the powerful-is vividly tangible

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

----------------------------------------------------------------------------------------------------
Progress: 10/10 | Query: The Wheel of Darkness (Pendergast)
----------------------------------------------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `search_database_for_book_information` with `{'query_text': 'The Wheel of Darkness Pendergast'}`




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [['The Wheel of Darkness (Agent Pendergast series, 8) - Taking Constance on an international tour to expose her to a world she has never experienced, FBI agent Pendergast visits the Tibetan monastery where he received martial arts training, only to learn that a rare and dangerous artifact has been stolen, following a trail that leads him aboard a treacherous Atlantic cruise. Reprint. -  Fiction , Thrillers , General', "The Wheel of Darkness (Pendergast) - FBI Special Agent Pendergast is taking a break from work to take Constance on a whirlwind Grand Tour, hoping to give her closure and a sense of the world that she's missed.  They head to Tibet, where Pendergast intensively trained in martial arts and spiritual studies. At a remote monastery, they learn that a rare and dangerous artifact the monks have been guarding for generations has been mysteriously stolen.  As a favor, Pendergast agrees to track and recover the relic.  A twisting trail of bloodshed leads Pendergast a

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]